In [29]:
import configparser
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objects as go

config = configparser.ConfigParser()
config.read('config.ini')
mapbox_token = config['mapbox']['secret_token']

#read file into dataframe 
df = pd.read_csv('covid-19-pandemic-worldwide-data-Canada.csv', sep=';')

#change all columns to lower case
df.columns = [col.lower() for col in df.columns]

lat = []
lon = []
for row in df['location']:
    try:
        lat.append(row.split(',')[0])
        lon.append(row.split(',')[1])
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)
df['lat'] = lat
df['lon'] = lon
#Saving provinces positions (latitude and longitude per subzones)
province_position = df[['sub zone', 'lat', 'lon']].drop_duplicates(['sub zone']).set_index(['sub zone'])

# Pivoting per category
df = pd.pivot_table(df, values='count', index=['date', 'sub zone'], columns=['category'])

df.columns = ['confirmed', 'deaths']

# Merging locations after pivoting
df = df.join(province_position)

# Compute bubble sizes
df['size'] = df['confirmed'].apply(lambda x: (np.sqrt(x/50) + 1) if x > 10 else (np.log(x) / 2 + 1)).replace(np.NINF, 0)

# Compute bubble color
df['color'] = df['confirmed'].fillna(0).replace(np.inf , 0)

# Selecting the day to display
day = '2020-11-30'
info = df.xs(day)

# Create the figure and feed it all the prepared columns
fig = go.Figure(
    go.Scattermapbox(
        lat=info['lat'],
        lon=info['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=info['size'],
            color=info['color']
        )
    )
)

# Specify layout information
fig.update_layout(
     autosize=True,
     mapbox_style='mapbox://styles/unity-19/ckikuhevt1aqx17l97tj8dv8u',
     mapbox=dict(
        accesstoken=mapbox_token, #
        center=go.layout.mapbox.Center(lat=64, lon=-100),
        zoom=2
    )
)

# Display the figure
fig.show()

#code have been modified from https://towardsdatascience.com/how-to-create-animated-scatter-maps-with-plotly-and-dash-f10bb82d357a